![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)


<p><font size="5" color='grey'> <b>
Agenten
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)

# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>

def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'SERPAPI_API_KEY': 'SERPAPI_API_KEY',
        'WEATHER_API_KEY': 'WEATHER_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        'GOOGLE_API_KEY': 'GOOGLE_API_KEY',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Grundlagen von KI-Agenten

<p><font color='black' size="5">
Was sind KI-Agenten?
</font></p>


**Definition und Abgrenzung zu anderen KI-Systemen:**    
KI-Agenten sind autonome Systeme, die ihre Umgebung wahrnehmen, Entscheidungen treffen und Aktionen ausführen können, um bestimmte Ziele zu erreichen. Im Gegensatz zu einfachen KI-Modellen, die nur auf Eingabe reagieren und Ausgabe produzieren, können Agenten proaktiv handeln und aus ihren Erfahrungen lernen.

**Autonomie, Reaktivität und zielgerichtetes Handeln:**    
Ein KI-Agent kann selbstständig (autonom) Entscheidungen treffen, auf Veränderungen in seiner Umgebung reagieren (Reaktivität) und hat ein oder mehrere Ziele, die sein Handeln leiten. Dabei verfolgt er langfristige Strategien statt nur auf unmittelbare Reize zu reagieren.

**Unterschied zwischen einfachen LLM-Anwendungen und Agenten:**    
Einfache LLM-Anwendungen (Large Language Models) wie ein Chatbot antworten lediglich auf Anfragen, während Agenten darüber hinaus eigene Entscheidungen treffen, Tools nutzen und zielgerichtet handeln können. Ein Agent kann beispielsweise entscheiden, wann er eine Suchanfrage starten sollte, um eine Frage besser zu beantworten.



# 2 | Agententypen
---


<p><font color='black' size="5">
Traditionelle Agententypen
</font></p>


Haupttypen von Agenten nach Anthropic:

- **Workflows**: Systeme mit vordefinierten, festen Abläufen (**Codepfaden**), bei denen LLMs und Tools nach einem **festen Plan** orchestriert werden. Die Entscheidungspfade sind vorab festgelegt und ändern sich nicht dynamisch während der Ausführung[.
- **Agenten**: Systeme, in denen LLMs ihre Prozesse und Tool-Nutzung **dynamisch** steuern, **flexibel** auf Situationen reagieren und eigenständig Aufgaben erfüllen können. Sie können **Entscheidungen treffen, planen, lernen** und sich an neue Situationen anpassen.

| Agententyp                       | Zuordnung Haupttyp (Anthropic) | Beschreibung                                                                            |
| :------------------------------- | :----------------------------- | :-------------------------------------------------------------------------------------- |
| Einfache Reflexionsagenten       | Workflow                       | Folgen starren Wenn-Dann-Regeln, keine Flexibilität oder dynamische Steuerung.          |
| Modellbasierte Reflexionsagenten | Workflow                       | Haben internes Modell, aber Abläufe sind weiterhin vorgegeben und nicht dynamisch.      |
| Zielbasierte Agenten             | Übergang Workflow → Agent      | Treffen Entscheidungen basierend auf Zielen, können aber noch festen Abläufen folgen.   |
| Nutzungsbasierte Agenten         | Agent                          | Bewerten Optionen, treffen differenzierte Entscheidungen, zeigen bereits adaptive Züge. |
| Lernende Agenten                 | Agent                          | Lernen aus Erfahrungen, passen Verhalten an, sind maximal flexibel und autonom.         |

- **Workflows** eignen sich für einfache bis mäßig komplexe Aufgaben, bei denen der Ablauf vorhersehbar und festgelegt ist. Das entspricht den einfachen und modellbasierten Reflexionsagenten, die keine oder nur sehr eingeschränkte Anpassungsfähigkeit besitzen[^3][^5].
- **Agenten** im Sinne von Anthropic sind Systeme, die flexibel, adaptiv und oft lernfähig sind. Sie treffen eigenständig Entscheidungen, planen und können ihr Verhalten anpassen. Das entspricht den nutzungsbasierten und lernenden Agenten aus der KI-Literatur[^2][^5].
- **Zielbasierte Agenten** liegen an der Schnittstelle: Sie können noch als Workflows umgesetzt werden, wenn die Zielerreichung einem festen Plan folgt. Sobald sie aber flexibel zwischen Zielen abwägen oder den Ablauf dynamisch anpassen, zählen sie zu den Agenten.


Diese Einteilung entspricht den klassischen Definitionen nach Russell & Norvig sowie Wooldridge & Jennings und wird in der Fachliteratur als Standard angesehen. Sie beschreibt die zunehmende Komplexität und Autonomie der Agenten und ist besonders geeignet, wenn die interne Funktionsweise, die Entscheidungsfindung und die Fähigkeit zur Anpassung im Vordergrund stehen.


<p><font color='black' size="5">
Aufgabenorientierte Agententypen
</font></p>



In der Praxis ist es oft sinnvoller, Agenten nach ihrem Anwendungszweck und ihrer funktionalen Ausrichtung zu klassifizieren:

| Agentenklasse | Primärer Zweck | Typische Anwendungen | Schlüsselmerkmale |
|---------------|----------------|----------------------|-------------------|
| **Problem-Solving Agent** | Komplexe Probleme lösen | Recherchen, analytische Aufgaben, mehrstufige Entscheidungen | - Kombiniert Reasoning mit Tool-Nutzung<br>- Explizites Denken und strukturierte Schritte<br>- Flexibilität bei verschiedenen Problemtypen |
| **Verbesserungs-Agent** | Inhaltsqualität optimieren | Content-Erstellung, Code-Review, Textkorrekturen | - Selbstbewertung und -kritik<br>- Iterative Verbesserung<br>- Vergleich von Versionen |
| **Service-Agent** | Strukturierte Dienste ausführen | API-Integrationen, Datenabfragen, automatisierte Aktionen | - Standardisierte Schnittstellen<br>- Klare Ein- und Ausgaben<br>- Zuverlässige Ausführung definierter Funktionen |
| **Spezialisierter Agent** | Domänenspezifische Aufgaben | Kundenservice, medizinische Beratung, rechtliche Informationen | - Tiefes Domänenwissen<br>- Angepasste Tools für spezifische Zwecke<br>- Optimiert für einen Anwendungsbereich |



**Wichtiger Hinweis zur Implementierung:**

Die in diesem Kurs gezeigten Implementierungen verwenden eine flexible Toolset-Architektur, die für verschiedene Agentenklassen genutzt werden kann. Beachten Sie, dass der Unterschied zwischen den Agentenklassen weniger in der technischen Implementierung liegt als in:

1. Der Aufgabenstellung und dem Anwendungsfall
2. Dem Verhaltensmuster des Agenten
3. Der Art, wie verschiedene Werkzeuge kombiniert werden

In der Praxis kombinieren moderne Agenten oft Elemente aus verschiedenen konzeptionellen Architekturen, um optimale Ergebnisse zu erzielen.



# 3 | Aufgabenorientierte Agenten
---




**Setup der Entwicklungsumgebung:** Installation und Konfiguration der notwendigen Bibliotheken, insbesondere LangChain und die Anbindung an ein LLM wie OpenAI.

**ToolSet definieren:** Definition der durch die Agents benutzten Tools.

**Implementierung eines einfachen Agenten:** Ein Beispiel für einen einfachen Agenten, der Fragen beantwortet und Berechnungen durchführt.




<p><font color='black' size="5">
3.1 Zentrale Tool-Bibliothek
</font></p>

In [ ]:
# Install
!uv pip install --system --prerelease allow -q langchain-openai langchain-community google-search-results wikipedia

Hier eine kurze Übersicht der nachfolgend definierten Tools:

| **Tool-/Funktion-Name**   | **Funktion**                                                                                |
| ------------------------- | ------------------------------------------------------------------------------------------- |
| `search`                  | Sucht nach Informationen im Internet über SerpAPI.                                          |
| `wiki`                    | Holt den ersten Wikipedia-Abschnitt zu einem Thema.                                         |
| `calculator`              | Führt einfache mathematische Berechnungen aus (z. B. 2+2 oder (3\*4)/2).                    |
| `weather`                 | Gibt das aktuelle Wetter für einen angegebenen Ort zurück (via OpenWeather).                |
| `read_file`               | Liest den Inhalt einer Datei aus (Pfadangabe erforderlich).                                 |
| `write_file`              | Schreibt Inhalt in eine Datei im Format `pfad//inhalt`.                                     |
| `days_between_dates`      | Berechnet die Anzahl der Tage zwischen zwei Datumsangaben im Format `DD.MM.YYYY`.           |
| `extract_math_expression` | Extrahiert einen mathematischen Ausdruck aus einem Text (z. B. aus „Was ist 3+5?" → `3+5`). |



In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Definition der Tools</font> </br></p>
# Import
import os
import re
import requests
import json
from datetime import datetime
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Tool-Funktionen definieren
def get_weather(location):
    """Gibt das aktuelle Wetter für einen bestimmten Ort zurück."""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return f"Aktuelle Temperatur in {location}: {data['main']['temp']}°C, {data['weather'][0]['description']}"

def read_file(file_path):
    """Liest den Inhalt einer Datei."""
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    return f"Fehler: Datei {file_path} nicht gefunden."

def write_file(args):
    """Schreibt Inhalt in eine Datei. Format: 'pfad//inhalt'"""
    try:
        file_path, content = args.split("//", 1)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"Erfolgreich in {file_path} geschrieben."
    except Exception as e:
        return f"Fehler beim Schreiben: {str(e)}"

def days_between_dates(date_string):
    """Berechnet Tage zwischen zwei Datumsangaben im Format DD.MM.YYYY"""
    try:
        date1_str, date2_str = date_string.split(",")
        date1_str = date1_str.strip()
        date2_str = date2_str.strip()
        date1 = datetime.strptime(date1_str, "%d.%m.%Y").date()
        date2 = datetime.strptime(date2_str, "%d.%m.%Y").date()
        return str((date2 - date1).days)
    except ValueError:
        return "Ungültiges Datumsformat. Bitte verwende DD.MM.YYYY."

def extract_math_expression(text):
    """Extrahiert mathematischen Ausdruck aus Text."""
    match = re.search(r'([\d+\-*/().]+)', text)
    return match.group(0) if match else None

def calculate(expression):
    """Führt mathematische Berechnungen durch."""
    try:
        math_expr = extract_math_expression(expression)
        if math_expr:
            return str(eval(math_expr))
        return "Keine gültige Berechnung gefunden."
    except:
        return "Fehler bei der Berechnung."

# Externe APIs
serpapi = SerpAPIWrapper()
wiki = WikipediaAPIWrapper()

# Tools-Liste, die in allen Beispielen verwendet werden kann
tools = [
    Tool(name="search", func=serpapi.run, description="Infos aus dem Internet"),
    Tool(name="wiki", func=lambda x: wiki.run(x).split("\n")[0] if wiki.run(x) else "Keine relevanten Infos gefunden.",
         description="Fakten aus Wikipedia"),
    Tool(name="calculator", func=calculate, description="Mathematische Berechnungen"),
    Tool(name="weather", func=get_weather, description="Gibt das aktuelle Wetter für einen bestimmten Ort zurück"),
    Tool(name="read_file", func=read_file, description="Liest den Inhalt einer Datei. Input sollte der Dateipfad sein."),
    Tool(name="write_file", func=write_file, description="Überschreibt eine Datei mit neuem Inhalt. Input sollte im Format 'pfad//inhalt' sein."),
    Tool(name="days_between_dates", func=days_between_dates, description="Berechnet die Anzahl der Tage zwischen zwei Datumsangaben im Format DD.MM.YYYY.")
]


<p><font color='black' size="5">
3.2 Aufbau eines Agenten
</font></p>



Erläuterung der einzelnen Funktionen:

* `ChatOpenAI` ermöglicht die Nutzung von OpenAI's Chat-Modellen.
* `create_tool_calling_agent` und `AgentExecutor` erschaffen und führen einen Agenten aus, der Werkzeuge nutzen kann.
* `ChatPromptTemplate` und `MessagesPlaceholder` definieren die Struktur für Nachrichten und Vorlagen im Chat.
* `IPython.display display, Markdown` ermöglichen die Darstellung von formatiertem Text in einer Jupyter-Umgebung.

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import display, Markdown

# Konstaten
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

# LLM initialisieren
llm = ChatOpenAI(model_name=MODEL, temperature=TEMPERATUR)

# Prompt erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent mit Zugang zu Tools. Bitte Formeln in $ Formel $ formatieren."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

Das `agent_scratchpad` ist ein entscheidender Bestandteil des Agenten-Frameworks, insbesondere bei der Verwendung von Werkzeugen (Tools):

* **Arbeitsbereich für den Agenten:**
    * Das `agent_scratchpad` dient als temporärer Arbeitsbereich, in dem der Agent seine Überlegungen, Aktionen und Beobachtungen während der Ausführung speichert.
    * Es ist der Ort, an dem der Agent seine "Denkschritte" festhält, bevor er eine endgültige Antwort gibt.
* **Kommunikation mit dem Sprachmodell:**
    * Wenn der Agent ein Werkzeug aufruft, werden die Details des Aufrufs (z. B. der Name des Werkzeugs und die Eingabeparameter) in den `agent_scratchpad` geschrieben.
    * Nachdem das Werkzeug ausgeführt wurde, wird das Ergebnis (die "Beobachtung") ebenfalls in den `agent_scratchpad` geschrieben.
    * Das Sprachmodell verwendet den Inhalt des `agent_scratchpad`, um zu entscheiden, welche nächsten Schritte es unternehmen soll (z. B. ein weiteres Werkzeug aufrufen oder eine endgültige Antwort geben).
* **Debugging und Transparenz:**
    * Das `agent_scratchpad` kann auch für Debugging-Zwecke nützlich sein, da er Einblicke in die Denkprozesse des Agenten bietet.
    * Es macht die Aktionskette des Agenten transparenter.

Im Kontext des Prompts:

* `MessagesPlaceholder(variable_name="agent_scratchpad")` fügt den Inhalt des `agent_scratchpad` in den Prompt ein, der an das Sprachmodell gesendet wird.
* Dadurch kann das Sprachmodell die vorherigen Aktionen und Beobachtungen des Agenten berücksichtigen, wenn es seine nächste Antwort generiert.

Zusammengefasst ist das `agent_scratchpad` ein dynamischer Bereich, der die Kommunikation und das Gedächtnis des Agenten während der Ausführung von Werkzeugen ermöglicht.

In [ ]:
# Agent erstellen und ausführen
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)

In [ ]:
# Anfrage stellen
response = agent_executor.invoke({
    "input": "Welche Tools stehen zu Deiner Verfügung?",
    "chat_history": []
})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🕵️‍♀️ Agent"))
display(Markdown("---"))
display(Markdown(response["output"]))

<p><font color='black' size="5">
Weitere Fragen an den Agenten
</font></p>

In [ ]:
# Anfrage stellen
response = agent_executor.invoke({
    "input": "Wie viel ist 12 * 12? Und wer ist Taylor Swift?",
    "chat_history": []
})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🕵️‍♀️ Agent"))
display(Markdown("---"))
display(Markdown(response["output"]))

In [ ]:
# Anfrage stellen
response = agent_executor.invoke({
    "input": "Schreibe eine Notiz zum Wetter in Essen in eine Datei mit dem Namen wetter.txt",
    "chat_history": []
})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🕵️‍♀️ Agent"))
display(Markdown("---"))
display(Markdown(response["output"]))

In [ ]:
# Anfrage stellen
response = agent_executor.invoke({
    "input": "Lese den Inhalt der Datei wetter.txt",
    "chat_history": []
})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🕵️‍♀️ Agent"))
display(Markdown("---"))
display(Markdown(response["output"]))


<p><font color='black' size="5">
3.3 Problem-Solving Agent
</font></p>


**Kernprinzip:** Zyklischer Ablauf von Denken, Handeln und Beobachten mit explizitem Reasoning.

**Charakteristika:**

- Explizite Gedankenformulierung vor jeder Aktion
- Beobachtung der Ergebnisse nach jeder Aktion
- Iterative Anpassung basierend auf Beobachtungen

**Historische Einordnung:**
Diese Implementierung folgt dem ReAct-Paradigma (Reasoning + Acting) und steht beispielhaft für die breitere Kategorie der Problem-Solving Agents.

In [ ]:
# Dieses Beispiel demonstriert einen Problem-Solving Agent, der eine komplexe
# Recherche-Aufgabe bewältigt. Beachten Sie, wie der Agent explizite
# Gedankengänge formuliert und sein Reasoning offenlegt. Diese Implementierung
# folgt historisch dem ReAct-Paradigma (Reasoning + Acting), ist aber ein
# Beispiel für die breitere Kategorie der Problem-Solving Agents.

In [ ]:
# Import
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from datetime import datetime
from IPython.display import display, Markdown

# Konstaten
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

In [ ]:
# LLM initialisieren
llm = ChatOpenAI(model_name=MODEL, temperature=TEMPERATUR)

In [ ]:
# Spezifischer Prompt aus LangChain Hub
prompt = hub.pull("hwchase17/react")
for p in prompt:
    print(p)

In [ ]:
# Agent erstellen
agent = create_react_agent(llm, tools, prompt)

# Agent Executor zur Ausführung verwenden
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [ ]:
# Beispiel für einen Problem-Solving Agent bei komplexen Recherche-Aufgaben
query = """
Welcher Schauspieler hat sowohl mit Christopher Nolan als auch mit Steven Spielberg gearbeitet?
Wie ist der Name des Schaupielers?
Wenn es mehrere Schauspieler gibt, dann nehme den ersten in Deiner Liste.
Welche Filme waren das und
In welchem Jahr wurden sie jeweils veröffentlicht?
Beschränke Dich auf meine Fragen.
"""

result = agent_executor.invoke({"input": query})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🔍 Problem-Solving Agent (ReAct) Ergebnis"))
display(Markdown("---"))
display(Markdown(result["output"]))


<p><font color='black' size="5">
3.4 Spezialisierter Agent
</font></p>



**Kernprinzip:** Kombination eines zentralen Sprachmodells mit spezialisierten Modulen für verschiedene Domänen.

**Charakteristika:**

- Modularität: Verschiedene Tools für spezialisierte Aufgaben
- Routing: Weiterleitung von Teilaufgaben an die passenden Tools
- Integration: Zusammenführung der Ergebnisse in eine kohärente Antwort

**Historische Einordnung:**
Diese Implementierung folgt dem MRKL-Paradigma (Modular Reasoning, Knowledge and Language) und repräsentiert die breitere Kategorie der Spezialisierten Agents.

In [ ]:
# Dieses Beispiel demonstriert einen Spezialisierten Agent, der verschiedene
# Aspekte einer Reiseplanung bewältigt. Die Implementierung zeigt, wie ein
# Agent verschiedene Tools für unterschiedliche Teilaufgaben einsetzen kann.
# Historisch folgt dieser Ansatz dem MRKL-Paradigma (Modular Reasoning, Knowledge
# and Language), repräsentiert aber die breitere Kategorie der Spezialisierten Agents.
# Wir können hier denselben AgentExecutor weiterverwenden, aber mit einer anderen Aufgabe

In [ ]:
# Beispiel für einen Spezialisierten Domänen-Agent (basierend auf MRKL)
query = """
Ich plane eine Reise nach Rom für die erste Juniwoche.
Wie wird das Wetter dort sein?
Welche historischen Stätten sollte ich besuchen?
Wie hoch sind die Eintrittsgelder in EURO?
Welche lokalen Gerichte sind empfehlenswert?
Wieviele Tage sind bis zur Abreise? Heute ist der 03.05.2025.
Beschränke Dich auf meine Fragen.
"""

# Agent ausführen
response = agent_executor.invoke({"input": query})

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🧩 Spezialisierter Agent Ergebnis"))
display(Markdown("---"))
display(Markdown(response["output"]))


<p><font color='black' size="5">
3.5 Verbesserungs-Agent
</font></p>



**Kernprinzip:** Selbstbewertung und -verbesserung durch kritische Analyse der eigenen Outputs.

**Charakteristika:**

- Self-criticism: Kritische Bewertung der eigenen Ausgaben
- Verbesserungszyklen: Mehrfache Iteration zur Qualitätsverbesserung
- Meta-Kognition: "Denken über das Denken"

**Historische Einordnung:**
Diese Implementierung basiert auf dem Konzept der Reflective Agents und repräsentiert die breitere Kategorie der Verbesserungs-Agents.

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import display, Markdown

In [ ]:
# Prompts für die drei Phasen des Reflection-Prozesses
answer_prompt = PromptTemplate.from_template("Beantworte: {question}")
reflection_prompt = PromptTemplate.from_template("""
Bewerte:
Frage: {question}
Antwort: {answer}

Fehler/Verbesserungen?""")
improvement_prompt = PromptTemplate.from_template("""
Frage: {question}
Antwort: {answer}
Reflexion: {reflection}

Verbesserte Antwort:""")

In [ ]:
# LCEL-Chains für die drei Phasen

# LLMs mit unterschiedlichen Einstellungen
answer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # Kreativere Erstantwort
reflection_llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Kritische Bewertung

# Chains answer, reflection, improvement
answer_chain = answer_prompt | answer_llm | StrOutputParser()

reflection_chain = {
    "question": lambda x: x["question"],
    "answer": lambda x: x["answer"]
} | reflection_prompt | reflection_llm | StrOutputParser()

improvement_chain = {
    "question": lambda x: x["question"],
    "answer": lambda x: x["initial_answer"],
    "reflection": lambda x: x["reflection"]
} | improvement_prompt | answer_llm | StrOutputParser()

In [ ]:
# Ausführung der Chains / des Verbesserungs-Agents
question = "Was sind die Hauptursachen des Klimawandels?"

initial_answer = answer_chain.invoke({"question": question})

reflection = reflection_chain.invoke({"question": question, "answer": initial_answer})

improved_answer = improvement_chain.invoke({
    "question": question,
    "initial_answer": initial_answer,
    "reflection": reflection
})

In [ ]:
# Ergebnisse anzeigen
display(Markdown("## 🔍 Verbesserungs-Agent (Reflective) Prozess"))
display(Markdown("---"))

display(Markdown("### Initiale Antwort:"))
display(Markdown(initial_answer))
display(Markdown("---"))

display(Markdown("### Selbstreflexion:"))
display(Markdown(reflection))
display(Markdown("---"))

display(Markdown("### Verbesserte Antwort:"))
display(Markdown(improved_answer))
display(Markdown("---"))


<p><font color='black' size="5">
3.6 Service-Agent
</font></p>




**Kernprinzip:** Standardisierte Schnittstellen für den Aufruf externer Funktionen und Dienste.

**Charakteristika:**
- Direkte Integration: Nahtlose Verbindung zu externen Funktionen
- Standardisierte Schnittstellen: Klare Definitionen für Ein- und Ausgaben
- Spezialisierte Aktionen: Fokus auf spezifische Funktionalitäten

**Historische Einordnung:**
Diese Implementierung folgt dem Function-Calling Paradigma und repräsentiert die breitere Kategorie der Service-Agents.

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import display, Markdown

# Konstaten
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

# LLM initialisieren
llm = ChatOpenAI(model=MODEL, temperature=TEMPERATUR)

In [ ]:
# Prompt für den Service-Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent, der Informationen über das Wetter und andere Fakten liefert."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
# Function-Calling Agent (fokussiert auf direkten Funktionsaufruf)
function_agent = create_openai_functions_agent(llm, tools, prompt)

service_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=function_agent,
    tools=tools,
    verbose=True
)

In [ ]:
# Typisches Service-Agent Beispiel
location = "Berlin"
response = service_agent_executor.invoke({
    "input": f"Wie ist das Wetter in {location}? Und was gibt es für allgemeine Informationen zu {location}?",
})

# Ergebnis anzeigen
display(Markdown("## 🌡️ Service-Agent (Function-Calling) Ergebnis"))
display(Markdown("---"))
display(Markdown(response["output"]))


<p><font color='black' size="5">
3.7 Vergleich der Agenten
</font></p>

|Aspekt|Problem-Solving Agent|Verbesserungs-Agent|Service-Agent|Spezialisierter Agent|
|---|---|---|---|---|
|Hauptfokus|Komplexe Probleme lösen|Inhaltsqualität optimieren|Dienste ausführen|Domänenexpertise|
|Arbeitsweise|Strukturierte Schritte mit Reasoning|Iterative Selbstbewertung|Standardisierte Schnittstellen|Spezialisierte Tools für ein Thema|
|Typischer Anwendungsfall|Recherche, Analyse, Entscheidungsfindung|Content-Erstellung, Review, QA|API-Interaktionen, Datenabfragen|Spezialisierte Beratung, Expertensysteme|
|Flexibilität|Hoch für verschiedene Probleme|Mittel, fokussiert auf Verbesserung|Begrenzt auf definierte Funktionen|Hoch innerhalb der Domäne|
|Transparenz|Hoch durch explizite Gedanken|Sehr hoch durch Selbsterklärung|Mittel|Variiert je nach Implementierung|
|Komplexitätsbewältigung|Vielseitig für komplexe Probleme|Gut für Qualitätsverbesserung|Gut für definierte Aufgaben|Hervorragend in der Fachdomäne|
|Implementation|Strukturierte Reasoning-Prozesse|Feedback-Schleifen|API-Definitionen|Domänenspezifische Tools|

Diese Tabelle bietet einen vollständigen Vergleich der aufgabenorientierten Agententypen in allen relevanten Aspekten und hilft bei der Auswahl des geeigneten Agententyps für bestimmte Anwendungsfälle.



<p><font color='black' size="5">
3.8 Kombinationsmöglichkeiten
</font></p>



Die Stärke moderner KI-Agenten liegt oft in der Kombination verschiedener Ansätze:

- **Problem-Solving + Service**: Verbindet strukturierte API-Aufrufe mit transparentem Reasoning
- **Spezialisiert + Verbesserung**: Domänenspezifische Systeme mit Selbstverbesserungsfähigkeit
- **Service + Spezialisiert**: Standardisierte Schnittstellen für modulare Systeme
- **Problem-Solving + Verbesserung**: Transparentes Reasoning mit iterativer Qualitätsverbesserung

In der Praxis implementieren die meisten Agenten eine Mischung dieser Ansätze, angepasst an spezifische Anwendungsfälle.


# 4 | Erweiterte Konzepte
---

Nach der Darstellung der grundlegenden Architekturen folgen nun erweiterte Konzepte, die auf diesen Architekturen aufbauen oder sie kombinieren.



<p><font color='black' size="5">
4.1 Einfache Planung
</font></p>



Planung und Zielverfolgung ist kein eigenständiger Architekturtyp, sondern ein Anwendungskonzept, das auf den grundlegenden Architekturen aufbaut.

**Kombination:** Meist basierend auf Problem-Solving oder Verbesserungs-Agents

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

# Konstaten
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

# LLM initialisieren
llm = ChatOpenAI(model_name=MODEL, temperature=TEMPERATUR)

In [ ]:
# Chain-of-Thought Prompt erstellen
cot_prompt = PromptTemplate(
    input_variables=["problem"],
    template="""
    Zerlege das folgende Problem in kleinere Teilschritte und löse es Schritt für Schritt:

    Problem: {problem}

    Denke schrittweise:
    1. Welche Teilprobleme müssen gelöst werden?
    2. Wie löse ich jeden Teilschritt?
    3. Wie kombiniere ich die Teillösungen zur Gesamtlösung?
    """
)

In [ ]:
# LCEL-Pipeline erstellen und ausführen
planning_chain = (
    {"problem": RunnablePassthrough()}
    | cot_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Beispielanwendung
complex_problem = """
Berechne die Gesamtkosten für einen Einkauf von 3 Büchern zu je 12,99 €,
2 Notizbüchern zu je 4,50 € und einem Stift für 2,25 €,
wenn auf Bücher 7% Mehrwertsteuer und auf andere Artikel 19% Mehrwertsteuer anfällt.
Verwende für Formeln das Format $ Formel $.
"""

solution = planning_chain.invoke(complex_problem)

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 📝 Einfache Planung mit Chain-of-Thought"))
display(Markdown("---"))
display(Markdown(solution))


<p><font color='black' size="5">
4.2 Fortgeschrittene Planung
</font></p>



**Kombination:** Oft Spezialisierte + Verbesserungs-Agents für hierarchische Planung

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

# Konstaten
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

# LLM initialisieren
llm = ChatOpenAI(model_name=MODEL, temperature=TEMPERATUR)

In [ ]:
# Funktionen
def create_planner_chain():
    """Erstellt eine Chain für die hierarchische Planung."""
    planer_prompt = PromptTemplate.from_template(
        """
        Erstelle einen hierarchischen Plan, um das folgende Ziel zu erreichen:

        Ziel: {objective}

        Teile den Plan in:
        1. Strategische Ziele (hohe Ebene)
        2. Taktische Schritte (mittlere Ebene)
        3. Konkrete Aktionen (detaillierte Ebene)

        Für jede konkrete Aktion gib an, welche Tools oder Ressourcen benötigt werden.
        """
    )
    return ({"objective": RunnablePassthrough()} | planer_prompt | llm | StrOutputParser())

def create_execution_chain():
    """Erstellt eine Chain für die Anpassung des Plans basierend auf Feedback."""
    execution_prompt = PromptTemplate.from_template(
        """
        Hier ist der aktuelle Plan:

        {plan}

        Basierend auf dem folgenden Feedback, passe den Plan an:

        Feedback: {feedback}

        Überarbeiteter Plan:
        """
    )
    return ({"plan": RunnablePassthrough(), "feedback": RunnablePassthrough()}
            | execution_prompt | llm | StrOutputParser())

In [ ]:
# Planungsketten erstellen
planner_chain = create_planner_chain()
execution_chain = create_execution_chain()

In [ ]:
# Beispiel ausführen
objective = "Organisiere eine zweitägige Konferenz für 100 Teilnehmer zum Thema KI-Ethik"
initial_plan = planner_chain.invoke(objective)

In [ ]:
# Feedback geben und Plan anpassen
feedback = "Der Budgetrahmen wurde auf 15.000 € reduziert, und wir müssen die Konferenz auf einen Tag verkürzen."
revised_plan = execution_chain.invoke({"plan": initial_plan, "feedback": feedback})

In [ ]:
# Ergebnisse anzeigen
display(Markdown("## 📔 Hierarchische Planung"))
display(Markdown("---"))

display(Markdown("### Initialer Plan"))
display(Markdown(initial_plan))
display(Markdown("---"))

display(Markdown("### Überarbeiteter Plan"))
display(Markdown(revised_plan))
display(Markdown("---"))


<p><font color='black' size="5">
4.3 Zielorientierter Agent
</font></p>



**Kombination:** Service-Agent + Problem-Solving Agent für zielgerichtete Aktionen

In [ ]:
# Import
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.agents.format_scratchpad import format_to_openai_functions

# Konstaten
MODEL = "gpt-4o"
TEMPERATUR = 0.0

# LLM initialisieren
llm = ChatOpenAI(model_name=MODEL, temperature=TEMPERATUR)

# Chat-Verlauf initialisieren
chat_history = []

In [ ]:
# Prompt erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher KI-Assistent. Antworte auf Deutsch."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# Agent erstellen
agent = create_openai_functions_agent(llm, tools, prompt)

# Agent-Executor konfigurieren
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
# Funktion für die Ausführung einer Konversation
def run_conversation(input_text):
    # Agent ausführen mit Zugriff auf den Chat-Verlauf
    result = agent_executor.invoke({
        "input": input_text,
        "chat_history": chat_history
    })

    # Chat-Verlauf aktualisieren
    chat_history.extend([
        HumanMessage(content=input_text),
        AIMessage(content=result["output"])
    ])

    return result["output"]

In [ ]:
# Komplexe Anfrage
complex_goal = """
Finde heraus, wer der aktuelle Bundeskanzler von Deutschland ist,
wann er geboren wurde und erstelle eine kurze Zusammenfassung
seiner politischen Karriere.
"""

In [ ]:
# Agent mit komplexer Anfrage aufrufen
result = run_conversation(complex_goal)

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🔎 Zielorientierter Agent"))
display(Markdown("---"))
display(Markdown(result))

In [ ]:
# Komplexe Anfrage
complex_goal = """
Finde heraus, ob er verheiratet ist und mit wem?
Stelle Informationen aus ihrer Biografie zusammen.
"""

In [ ]:
# Agent mit komplexer Anfrage aufrufen
result = run_conversation(complex_goal)

In [ ]:
# Ergebnis anzeigen
display(Markdown("## 🔎 Zielorientierter Agent"))
display(Markdown("---"))
display(Markdown(result))


<p><font color='black' size="5">
4.4 Kollaborative Analyse
</font></p>



**Kombination:** Spezialisierte Agenten mit verschiedenen Schwerpunkten

In [ ]:
# Import
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Pipelines mit optimierten Modellen und Temperaturen

research_pipeline = PromptTemplate.from_template(
    """Du bist ein Recherche-Spezialist. Sammle Informationen zum Thema: {topic}"""
) | ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # Gute Balance zwischen Kreativität und Fakten


critique_pipeline = PromptTemplate.from_template(
    """Du bist ein kritischer Analyst. Bewerte folgende Recherche: {research}"""
) | ChatOpenAI(model="gpt-4o", temperature=0)  # Maximale Präzision für Analyse


synthesis_pipeline = PromptTemplate.from_template(
    """Du bist ein Informationssynthetisierer. Erstelle eine Zusammenfassung:
    Recherche: {research}
    Kritik: {critique}"""
) | ChatOpenAI(model="gpt-4o", temperature=0.2)  # Verbesserte Qualität für komplexe Zusammenfassung


decision_pipeline = PromptTemplate.from_template(
    """Du bist ein Entscheidungsträger. Basierend auf: {synthesis}
    Beantworte: {question}"""
) | ChatOpenAI(model="gpt-4o", temperature=0)  # Konsistente Entscheidungsfindung

In [ ]:
# Schrittweise Analyse definieren
def collaborative_analysis(topic, question):
    """Kollaborative Analyse mit LCEL-Pipelines."""
    # Prozess ausführen
    research = research_pipeline.invoke({"topic": topic})
    display(Markdown("### ✅ Recherche abgeschlossen"))

    critique = critique_pipeline.invoke({"research": research})
    display(Markdown("### ✅ Kritische Analyse abgeschlossen"))

    synthesis = synthesis_pipeline.invoke({"research": research, "critique": critique})
    display(Markdown("### ✅ Synthese abgeschlossen"))

    decision = decision_pipeline.invoke({"synthesis": synthesis, "question": question})
    display(Markdown("### ✅ Entscheidung getroffen"))

    return {
        "research": research,
        "critique": critique,
        "synthesis": synthesis,
        "decision": decision
    }

In [ ]:
# Beispielanwendung
topic = "Künstliche Intelligenz in der Gesundheitsversorgung"
question = "Sollten Krankenhäuser KI-Systeme zur Diagnoseunterstützung einsetzen?"

In [ ]:
# Multi-Agenten-Analyse ausführen
display(Markdown("## 👥 Multi-Agenten-System: Kollaborative Analyse"))
display(Markdown("---"))
results = collaborative_analysis(topic, question)

In [ ]:
# Finale Entscheidung anzeigen
display(Markdown(f"## 📑 Finale Entscheidung"))
display(Markdown("---"))

for step, txt  in results.items():
    display(Markdown(f"### {step.upper()}"))
    display(Markdown(f"{txt.content}"))
    display(Markdown("---"))

# 5 | Model Context Protocol (MCP)
---



**Wichtige Ressourcen:**

[Anthropic MCP](https://www.anthropic.com/news/model-context-protocol)

[OpenAI MCP](https://openai.github.io/openai-agents-python/mcp/)

[MCPServer](https://github.com/modelcontextprotocol/servers)




<p><font color='black' size="5">
5.1 Was ist MCP?
</font></p>


Ein Protokoll ist ein Regelwerk, das bestimmt, wie zwei Systeme miteinander kommunizieren. Protokolle regeln die Datenübertragung in Computernetzwerken, bei der Internetkommunikation und zwischen Softwaresystemen.

**Zum Beispiel:**

+ HTTP (Hypertext Transfer Protocol): Ermöglicht Websites die Kommunikation mit Browsern.
+ TCP/IP (Transmission Control Protocol/Internetprotokoll): Definiert, wie Datenpakete im Internet geroutet werden.
+ JSON-RPC (Aufrufen von Remoteprozeduren): Ein Protokoll, das den Datenaustausch im JSON-Format ermöglicht.    

Das **Model Context Protocol (MCP)** ist ein offenes Protokoll, das es großen Sprachmodellen (LLMs) ermöglicht, sich auf standardisierte Weise in externe Datenquellen und Tools zu integrieren. Dieses von Anthropic entwickelte Protokoll macht es KI-Modellen leicht, nahtlos mit einer Vielzahl von Tools und Datenquellen zusammenzuarbeiten.

Es soll die Interaktion von KI-Modellen, insbesondere Large Language Models (LLMs) und autonomen Agenten, mit externen Datenquellen und Tools zu **standardisieren und zu vereinfachen**. Ziel ist es, einen **einheitlichen Rahmen** zu schaffen, der es KI-Agenten ermöglicht, auf strukturierte Daten aus verschiedenen Quellen wie Datenbanken, APIs, Cloud-Speicher und Unternehmensanwendungen auf standardisierte Weise zuzugreifen, diese zu verarbeiten und darauf zu reagieren, **ohne dass für jede Quelle spezifische API-Integrationen erforderlich sind**.




<p><font color='black' size="5">
5.2 Warum wurde MCP entwickelt?
</font></p>



Die Notwendigkeit von MCP ergibt sich aus den **Ineffizienzen und Herausforderungen** aktueller KI-API-Interaktionen. Derzeit ist der Aufbau von KI-Agenten, die Daten aus verschiedenen Quellen abrufen, **fragmentiert, repetitiv und schwer zu skalieren**. Jedes Tool spricht seine eigene Sprache und erfordert **individuelle Integrationen**. MCP zielt darauf ab, diese Komplexität zu reduzieren und den **Entwicklungsaufwand zu minimieren**.




<p><font color='black' size="5">
5.3 Wie funktioniert MCP?
</font></p>



MCP basiert auf einer **Client-Server-Architektur**.

*   **MCP-Clients** sind typischerweise KI-Agenten, Anwendungen oder Systeme, die strukturierte Daten benötigen. Beispiele hierfür sind Claude Desktop, Cursor, Windsurf und Frameworks wie Langchain und Pydantic AI.
*   **MCP-Server** fungieren als **Vermittler**, die Daten von verschiedenen APIs, Datenbanken oder Unternehmenssystemen abrufen und diese in einem **einheitlichen Format** an die Clients zurückgeben.

Der Interaktionsprozess folgt einem strukturierten **Anfrage-Antwort-Zyklus**:

1.  Der KI-Agent (Client) sendet eine **strukturierte Anfrage** an den MCP-Server, in der die benötigten Daten oder die auszuführende Aktion in einem standardisierten Format definiert sind.
2.  Der MCP-Server **verarbeitet** diese Anfrage, authentifiziert sie, prüft die Berechtigungen und ermittelt, welche externen Systeme abgefragt werden müssen.
3.  Die eigentlichen **Datenabfragen** an die verschiedenen Quellen können parallel erfolgen.
4.  Die **Antworten** der verschiedenen Quellen werden vom MCP-Server in einem **einheitlichen, strukturierten Format standardisiert**, das für KI-Modelle leicht zu verarbeiten ist.

Ein wesentliches Konzept von MCP ist die **Reflection**. Dies bedeutet, dass ein MCP-Client einen MCP-Server nach seinen **verfügbaren Tools und Ressourcen** fragen kann, ohne vorherige Kenntnisse dieser Schnittstellen zu benötigen.

[Interaktive Visualisierung des MCP-Prozesses](https://claude.site/artifacts/1ba26344-3819-427d-9080-98381785f8df)




<p><font color='black' size="5">
5.4 Kernkonzepte
</font></p>



*   **Tools:** Funktionen, die das Modell nutzen kann, um Aktionen durchzuführen (z. B. Websuchen, Datenbankabfragen).
*   **Ressourcen:** Anhänge oder Daten, die dem Modell zur Verfügung gestellt werden (z. B. Dateien, Datenbankinhalte).
*   **Prompts:** Vorlagen, die Clients für Anfragen an das Modell verwenden können.




<p><font color='black' size="5">
5.5 Vorteile von MCP
</font></p>


*   **Vereinfachte Integrationen** und **reduzierte Komplexität**.
*   **Verbesserte Skalierbarkeit** und **Wiederverwendbarkeit** von Integrationen.
*   **Erhöhte Interoperabilität** zwischen KI-Modellen und externen Systemen.
*   **Zeitersparnis für Entwickler** durch weniger benutzerdefinierte API-Implementierungen.
*   Potenzial für eine **zentrale Authentifizierung** (zukünftig).

[Interaktive Visualisierung](https://claude.site/artifacts/1ba26344-3819-427d-9080-98381785f8df)

<p><font color='black' size="5">
Code-Beispiel zur Interaktion
</font></p>

In [ ]:
import json
import uuid
from typing import Dict, Any

class MCPClient:
    """Einfacher MCP-Client für die Kommunikation mit MCP-Servern."""

    def __init__(self, client_id: str, api_key: str):
        self.client_id = client_id
        self.api_key = api_key
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Client-ID": client_id
        }

    def create_request(self, tool_name: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
        """Erstellt eine strukturierte MCP-Anfrage."""
        request_data = {
            "protocol_version": "1.0",
            "request_id": str(uuid.uuid4()),
            "tool": {
                "name": tool_name,
                "parameters": parameters
            }
        }
        return request_data

    def send_request(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Sendet eine MCP-Anfrage an den MCP-Server."""
        # In einer realen Implementierung würde hier ein HTTP-Request erfolgen
        return {"status": "success", "result": {}}  # Platzhalter


class MCPServer:
    """Simulierter MCP-Server zum Testen der Interaktion."""

    def __init__(self):
        self.registered_tools = {
            "weather_info": self._get_weather_data,
            "database_query": self._execute_database_query,
            "text_translation": self._translate_text
        }

    def validate_request(self, request: Dict[str, Any]) -> bool:
        """Überprüft, ob die Anfrage dem MCP-Protokoll entspricht."""
        required_fields = ["protocol_version", "request_id", "tool"]
        if not all(field in request for field in required_fields):
            return False

        if "name" not in request["tool"] or request["tool"]["name"] not in self.registered_tools:
            return False

        return True

    def process_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """Verarbeitet eine validierte MCP-Anfrage."""
        if not self.validate_request(request):
            return {"error": "Ungültige Anfrage", "status": "error"}

        tool_name = request["tool"]["name"]
        parameters = request["tool"].get("parameters", {})

        # Tool ausführen
        tool_func = self.registered_tools[tool_name]
        result = tool_func(parameters)

        # Antwort standardisieren
        response = {
            "request_id": request["request_id"],
            "status": "success",
            "result": result
        }
        return response

    def _get_weather_data(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert den Abruf von Wetterdaten von einem externen Dienst."""
        return {
            "city": params.get("city", "Berlin"),
            "temperature": 22,
            "condition": "sonnig",
            "humidity": 65
        }

    def _execute_database_query(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert eine Datenbankabfrage."""
        return {
            "rows": [
                {"id": 1, "name": "Produkt A", "price": 29.99},
                {"id": 2, "name": "Produkt B", "price": 49.99}
            ],
            "total_rows": 2
        }

    def _translate_text(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert einen Übersetzungsdienst."""
        target_lang = params.get("target_language", "en")

        translations = {
            "en": "Hello world",
            "fr": "Bonjour le monde",
            "de": "Hallo Welt"
        }

        return {
            "original_text": params.get("text", ""),
            "translated_text": translations.get(target_lang, "Übersetzung nicht verfügbar"),
            "target_language": target_lang
        }


class LLMApplication:
    """LLM-Anwendung, die den MCP-Client verwendet."""

    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client

    def process_user_query(self, query: str) -> str:
        """Verarbeitet eine Nutzeranfrage mit MCP für zusätzliche Daten."""
        if "wetter" in query.lower():
            # Extrahiere Stadt (vereinfacht)
            city = "Berlin"

            # Erstelle und sende MCP-Anfrage
            request = self.mcp_client.create_request("weather_info", {"city": city})
            response = self.mcp_client.send_request(request)

            if response.get("status") == "success":
                weather_data = response.get("result", {})
                return self._format_weather_response(weather_data)
            return "Entschuldigung, ich konnte keine Wetterdaten abrufen."

        elif "übersetze" in query.lower():
            text = "Hallo, wie geht es dir?"
            target_lang = "en"

            request = self.mcp_client.create_request("text_translation", {
                "text": text,
                "target_language": target_lang
            })
            response = self.mcp_client.send_request(request)

            if response.get("status") == "success":
                translation_data = response.get("result", {})
                return self._format_translation_response(translation_data)
            return "Entschuldigung, ich konnte den Text nicht übersetzen."

        return "Ich verstehe Ihre Anfrage. Wie kann ich Ihnen helfen?"

    def _format_weather_response(self, weather_data: Dict[str, Any]) -> str:
        """Formatiert Wetterdaten für die Anzeige."""
        return (f"Das aktuelle Wetter in {weather_data.get('city')}: "
                f"{weather_data.get('temperature')}°C, {weather_data.get('condition')}.")

    def _format_translation_response(self, translation_data: Dict[str, Any]) -> str:
        """Formatiert Übersetzungsdaten für die Anzeige."""
        return (f"Übersetzung ({translation_data.get('target_language')}): "
                f"{translation_data.get('translated_text')}")


def run_example():
    """Führt ein Beispiel der MCP-Interaktion aus."""
    print("=== MCP-Interaktionsbeispiel ===\n")

    # MCP-Server initialisieren
    server = MCPServer()

    # MCP-Client initialisieren
    client = MCPClient(
        client_id="llm_app_123",
        api_key="sk_test_12345"
    )

    # LLM-Anwendung initialisieren
    llm_app = LLMApplication(client)

    # Mock für send_request
    def mock_send_request(request_data):
        print("\n[1] LLM-Anwendung erstellt MCP-Anfrage:")
        print(json.dumps(request_data, indent=2))

        print("\n[2] MCP-Client sendet Anfrage an MCP-Server")

        print("\n[3] MCP-Server validiert die Anfrage")
        valid = server.validate_request(request_data)
        print(f"Anfrage ist gültig: {valid}")

        print("\n[4] MCP-Server verarbeitet die Anfrage und kommuniziert mit externen Systemen")
        response = server.process_request(request_data)

        print("\n[5] Externe Systeme liefern Daten an MCP-Server")

        print("\n[6] MCP-Server standardisiert die Antwort:")
        print(json.dumps(response, indent=2))

        print("\n[7] MCP-Server sendet Antwort an MCP-Client")

        print("\n[8] MCP-Client stellt Kontext der LLM-Anwendung zur Verfügung")

        return response

    # Überschreiben der Methode
    client.send_request = mock_send_request

    # Beispiel-Nutzeranfrage
    query = "Wie ist das Wetter in Berlin?"

    print(f"\nNutzer fragt: '{query}'")
    print("\n[0] LLM-Anwendung analysiert die Nutzeranfrage")

    # LLM verarbeitet die Anfrage und nutzt MCP
    response = llm_app.process_user_query(query)

    print("\n[9] LLM-Anwendung nutzt den Kontext zur Beantwortung")
    print(f"\nAntwort an den Nutzer: '{response}'")


if __name__ == "__main__":
    run_example()

<p><font color='black' size="5">
Fazit
</font></p>





Das Model Context Protocol (MCP) stellt einen vielversprechenden Ansatz dar, um die Integration von KI-Modellen mit der Außenwelt zu **standardisieren und zu vereinfachen**. Durch die Definition eines gemeinsamen Protokolls und die Bereitstellung von Konzepten wie Tools und Ressourcen ermöglicht MCP die Entwicklung **flexiblerer, skalierbarer und wiederverwendbarer** KI-Anwendungen. Obwohl es noch Herausforderungen zu bewältigen gibt, hat MCP das Potenzial, die Art und Weise, wie KI-Agenten mit Daten und Tools interagieren, **grundlegend zu verändern**.

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Kalkulation
</font></p>

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und erstellen Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...

